In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchsummary

import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
torch.cuda.is_available() #check for CUDA

True

In [3]:
# Load the training data
batch_size = 256
num_workers = 12

train_data = torchvision.datasets.MNIST('../data/raw', train=True, download=True, transform=torchvision.transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

# Load the test data
test_data = torchvision.datasets.MNIST('../data/raw', train=False, download=True, transform=torchvision.transforms.ToTensor())
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)

In [4]:
# # Access the first image and its label
# image, label = train_data[0]  # You can change the index to view different images

# # Display the image and its label
# plt.imshow(image.squeeze(), cmap='gray')  # .squeeze() removes 1-dimensions, cmap='gray' shows the image in grayscale
# plt.title(f'Label: {label}')
# plt.show()

In [5]:
class mnistConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # Output: 32 x 28 x 28
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) # Output: 64 x 28 x 28
        self.pool1 = nn.MaxPool2d(2, 2)                                    # Output: 64 x 14 x 14
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)# Output: 128 x 14 x 14
        self.pool2 = nn.MaxPool2d(2, 2)                                    # Output: 128 x 7 x 7
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=0)# Output: 256 x 5 x 5
        self.conv5 = nn.Conv2d(256, 10, kernel_size=5, stride=1, padding=0) # Output: 10 x 1 x 1

        # Move all model parameters to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = F.relu(self.conv3(x))
        x = self.pool2(x)
        x = F.relu(self.conv4(x))
        x = self.conv5(x)
        x = x.view(-1, 10)  # Flatten to [batch_size, num_classes]
        return x
        
net = mnistConvNet()

criterion = torch.nn.CrossEntropyLoss() #Loss function
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #Optimizer
track_loss = []

for epoch in tqdm(range(100)):  # loop over the dataset multiple times ### Try -> Load image/batch by hand and put it in model
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Move inputs and labels to the same device as the model
        inputs = inputs.to(net.device)
        labels = torch.tensor(labels, device = net.device)
        labels = labels.to(net.device)

        #print(f"Type of inputs: {type(inputs)}, Type of labels: {type(labels)}") 
        #print(f"Shape of inputs: {inputs.shape}, Shape of labels: {labels.shape}")
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
            track_loss.append([epoch, running_loss])

print('Finished Training')

  0%|                                                   | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_6707/3559692956.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, device = net.device)
100%|█████████████████████████████████████████| 100/100 [11:36<00:00,  6.97s/it]

Finished Training


In [6]:
PATH = '../models/mnist_model.pth'
torch.save(net.state_dict(), PATH)

In [8]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        
        images = images.to(net.device)
        labels = torch.tensor(labels, device = net.device)
        labels = labels.to(net.device)
        
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

/tmp/ipykernel_6707/3133354368.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, device = net.device)


Accuracy of the network on the 10000 test images: 98 %
